In [1]:
import os
import requests
import json
import pandas as pd

In [2]:
# set folders
RESULTS_FOLDER = '.\\Results'

if not os.path.exists(RESULTS_FOLDER):
    os.makedirs(RESULTS_FOLDER)

In [3]:
# get request
# see API at the bottom of https://alternative.me/crypto/fear-and-greed-index/
page=requests.get('https://api.alternative.me/fng/?limit=0&date_format=world')

# convert to json and extract to DataFrame
data=json.loads(page.content)['data']
df_download=pd.DataFrame()
for item in data:
    df_download=pd.concat([df_download, pd.DataFrame({'date': item['timestamp'], 'index': item['value'],
                                                        'classification': item['value_classification']}, index=[0])])
df_download['date']=pd.to_datetime(df_download['date'], infer_datetime_format=True)

In [4]:
df_download

,date,index,classification
0,2023-04-24,53,Neutral
0,2023-04-23,56,Greed
0,2023-04-22,53,Neutral
0,2023-04-21,50,Neutral
0,2023-04-20,52,Neutral
...,...,...,...
0,2018-02-05,11,Extreme Fear
0,2018-02-04,24,Extreme Fear
0,2018-02-03,40,Fear
0,2018-02-02,15,Extreme Fear


In [5]:
df_download.to_csv(os.path.join(RESULTS_FOLDER, '99_Crypto_Fear_and_Greed_Index.csv'), index=False, sep=';')

# Alternative way to scrape directly from ChartJS

In [2]:
from seleniumwire import webdriver
from seleniumwire.utils import decode
import json
import time
import pandas as pd

CHROMEDRIVER_PATH = r"C:\Users\Alessandro Bitetto\Downloads\UniPV\ICOs\WebDriver\chromedriver"

In [ ]:
url='https://alternative.me/crypto/fear-and-greed-index/'
requested_item_url='https://alternative.me/api/crypto/fear-and-greed-index/history'

# have a look at https://www.youtube.com/watch?v=i9N_LrnDUnY&ab_channel=StevesieData from minute 2:00 to understand
# how to locate the url of the chart you want to extract. It will be "requested" and the Network tab in inspect mode
# will record the interaction and then you can extract the response. Otherwise see the first example in 
# https://pypi.org/project/selenium-wire/ to get the full list of Network interactions

In [ ]:
# load page and scroll down (to load the chart)
driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH)
driver.get(url)
time.sleep(3)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)

# press "Max" button in the chart
driver.find_element("xpath", "/html/body/div/main/section/div/div[5]/div[2]/div/span[5]").click()

# loop request and extract the chart data as json
downloaded_data=[]
for request in driver.requests:
    if request.response:
        if request.url == requested_item_url:
            print(f'Found: {request.url}')
            body = decode(request.response.body, request.response.headers.get('Content-Encoding', 'identity'))
            downloaded_data.append(json.loads(body))
print(f'\nTotal data downloaded: {len(downloaded_data)}')

In [ ]:
# convert json to DataFrame
converted_dataframe=[]
for data in downloaded_data:
    converted_dataframe.append(pd.DataFrame({'dates': data['data']['labels'], 'index': data['data']['datasets'][0]['data']}))